In [1]:
import pandas as pd
from pybedtools import BedTool
from pybedtools.featurefuncs import TSS
import os

In [2]:
import sys
args = sys.argv  # a list of the arguments provided (str)
print("processing distanceBased", args)
rawBed= args[1]

processing distanceBased ['/storage/bin/anaconda3/envs/yudeep/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/home/yudeep_rajbhandari1@baylor.edu/.local/share/jupyter/runtime/kernel-870e30c0-c92e-4a74-89d9-7744b4133e12.json']


In [3]:
def getAllgeneWithinDistance(enhancer,gene,distance):
    snp = BedTool(enhancer)
    gene = BedTool(gene)
    dfSNP = pd.read_csv(enhancer, sep='\t', header=None)
    win = snp.window(gene,w=10000,output='tempDistance.bed')
    df1 = pd.read_csv('tempDistance.bed', sep='\t', header=None)
    df1.drop(columns=df1.columns[:len(dfSNP.columns)], 
        axis=1, 
        inplace=True)
    df1.to_csv('result.tsv', sep="\t")
    
    os.remove('tempDistance.bed')

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
def getClosestgeneWithinDistance(enhancer,genes,distance,num):
    snp = BedTool(enhancer)
    gene = BedTool(genes)
    gene.sort()  
    nearby = snp.closest(gene, d=True,k=num,io=True, output='tempDistance.bed')
    dfSNP = pd.read_csv(enhancer, sep='\t', header=None)
    dfGene = pd.read_csv(genes, sep='\t', header=None)
    df1 = pd.read_csv('tempDistance.bed', sep='\t', header=None)
    df1 = df1[(df1[len(df1.columns)-1].between(-distance, distance))]

    
    df1.drop(columns=df1.columns[4:len(dfSNP.columns)], 
        axis=1, 
        inplace=True)
    df1.drop(columns=df1.columns[8:len(dfGene.columns)+4], 
        axis=1, 
        inplace=True)
    os.remove('tempDistance.bed')
    return df1

In [6]:
df32 = pd.read_csv('mart_export.txt.gz',sep="\t")

In [7]:
df32.head()

,Gene stable ID,Transcript stable ID,HGNC ID,HGNC symbol,Gene start (bp),Gene end (bp),Chromosome/scaffold name
0,ENSG00000227568,ENST00000458433,HGNC:39634,SNX18P26,18807279,18808018,13
1,ENSG00000230931,ENST00000446657,HGNC:39676,GXYLT1P1,18724415,18724904,13
2,ENSG00000204718,ENST00000330492,HGNC:39834,CNN2P12,18726706,18727414,13
3,ENSG00000231238,ENST00000652806,HGNC:42667,LINC00349,18538880,18551214,13
4,ENSG00000231238,ENST00000448748,HGNC:42667,LINC00349,18538880,18551214,13


In [8]:
def startPosition(df):
    return 'chr'+str(df['Chromosome/scaffold name'])

In [9]:
def startPosition(df):
    return 'chr'+str(df['Chromosome/scaffold name'])

In [10]:
df32['0'] = df32.apply(startPosition,axis=1)
df32['1'] = df32['Gene start (bp)']
df32['2'] = df32['Gene end (bp)']
df32['3'] = df32['Gene stable ID']

In [11]:
df32.tail()

,Gene stable ID,Transcript stable ID,HGNC ID,HGNC symbol,Gene start (bp),Gene end (bp),Chromosome/scaffold name,0,1,2,3
242216,ENSG00000117682,ENST00000525410,HGNC:20603,DHDDS,26432282,26471306,1,chr1,26432282,26471306,ENSG00000117682
242217,ENSG00000117682,ENST00000430232,HGNC:20603,DHDDS,26432282,26471306,1,chr1,26432282,26471306,ENSG00000117682
242218,ENSG00000117682,ENST00000703264,HGNC:20603,DHDDS,26432282,26471306,1,chr1,26432282,26471306,ENSG00000117682
242219,ENSG00000117682,ENST00000431933,HGNC:20603,DHDDS,26432282,26471306,1,chr1,26432282,26471306,ENSG00000117682
242220,ENSG00000117682,ENST00000416052,HGNC:20603,DHDDS,26432282,26471306,1,chr1,26432282,26471306,ENSG00000117682


In [12]:
df21 = df32[['0','1','2','3']]

In [13]:
df21.head()

,0,1,2,3
0,chr13,18807279,18808018,ENSG00000227568
1,chr13,18724415,18724904,ENSG00000230931
2,chr13,18726706,18727414,ENSG00000204718
3,chr13,18538880,18551214,ENSG00000231238
4,chr13,18538880,18551214,ENSG00000231238


In [14]:
df22 = df21.drop_duplicates(
  subset = ['0','1','2'],
  keep = 'last').reset_index(drop = True)

In [15]:
df22.tail()

,0,1,2,3
58767,chr1,439870,440232,ENSG00000269732
58768,chr1,450740,451678,ENSG00000284733
58769,chr1,487101,489906,ENSG00000233653
58770,chr1,491225,493241,ENSG00000250575
58771,chr1,26432282,26471306,ENSG00000117682


In [16]:
df21 = df21.sort_values(by=['0','1'],ascending=True)

In [17]:
new_header = df21.iloc[0] 
df21 = df21[1:] 
df21.columns = new_header 


In [18]:
df21.to_csv('temp21.bed', sep="\t",index=False)

In [19]:
#closestGene = getClosestgeneWithinDistance('ENCFF979PSS.bed.gz','../../../storage/data/dna/human/hg38/hg38.trf.bed.gz',1000000,1)

In [20]:
closestGene1 = getClosestgeneWithinDistance('ENCFF979PSS.bed.gz','temp21.bed',1000000,1)
# closestGene1 = getClosestgeneWithinDistance(rawBed,'temp21.bed',1000000,1)

In [21]:
df22 = pd.read_csv('../../../storage/data/dna/human/hg38/hg38.trf.bed.gz',sep="\t",header=None)

In [22]:
df22.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,chr1,10000,10468,trf,6,77.2,6,95,3,789,33,51,0,15,1.43,TAACCC
1,chr1,30862,30959,trf,2,47.5,2,75,8,79,5,42,1,51,1.31,TC
2,chr1,44835,44876,trf,4,10.0,4,94,5,73,73,0,0,26,0.84,AAAT
3,chr1,50481,50513,trf,2,16.0,2,100,0,64,0,0,50,50,1.00,GT
4,chr1,54712,54817,trf,4,25.8,4,87,7,149,0,26,0,72,0.91,TTTC


In [23]:
df22[(df22[0] == 'chr1') & (df22[1]=='147629652') & (df22[2]=='147670496')]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15


In [24]:
# closestGene.to_csv('result.tsv', sep="\t")

In [25]:
# closestGene.to

In [26]:
closestGene1.tail()

,0,1,2,3,11,12,13,14,15
4481558,chrY,26409363,26409710,EH38E3951258,.,-1,-1,.,-1
4481559,chrY,26563051,26563261,EH38E3951263,.,-1,-1,.,-1
4481560,chrY,26661142,26661310,EH38E3951264,.,-1,-1,.,-1
4481561,chrY,26670948,26671287,EH38E3951269,.,-1,-1,.,-1
4481562,chrY,26671292,26671478,EH38E3951270,.,-1,-1,.,-1


In [27]:
df32[(df32['Gene start (bp)'] ==147629652) & (df32['Gene end (bp)'] == 147670524) ]

,Gene stable ID,Transcript stable ID,HGNC ID,HGNC symbol,Gene start (bp),Gene end (bp),Chromosome/scaffold name,0,1,2,3
163310,ENSG00000162836,ENST00000609196,HGNC:29609,ACP6,147629652,147670524,1,chr1,147629652,147670524,ENSG00000162836
163311,ENSG00000162836,ENST00000583509,HGNC:29609,ACP6,147629652,147670524,1,chr1,147629652,147670524,ENSG00000162836
163312,ENSG00000162836,ENST00000460583,HGNC:29609,ACP6,147629652,147670524,1,chr1,147629652,147670524,ENSG00000162836
163313,ENSG00000162836,ENST00000613673,HGNC:29609,ACP6,147629652,147670524,1,chr1,147629652,147670524,ENSG00000162836
163314,ENSG00000162836,ENST00000487562,HGNC:29609,ACP6,147629652,147670524,1,chr1,147629652,147670524,ENSG00000162836
163315,ENSG00000162836,ENST00000392988,HGNC:29609,ACP6,147629652,147670524,1,chr1,147629652,147670524,ENSG00000162836
163316,ENSG00000162836,ENST00000620634,HGNC:29609,ACP6,147629652,147670524,1,chr1,147629652,147670524,ENSG00000162836
163317,ENSG00000162836,ENST00000611629,HGNC:29609,ACP6,147629652,147670524,1,chr1,147629652,147670524,ENSG00000162836
163318,ENSG00000162836,ENST00000614551,HGNC:29609,ACP6,147629652,147670524,1,chr1,147629652,147670524,ENSG00000162836
163319,ENSG00000162836,ENST00000493129,HGNC:29609,ACP6,147629652,147670524,1,chr1,147629652,147670524,ENSG00000162836


### Duplication issue
Multiple genes with same gene stable ID , gene start and gene end but different Transcript stable ID

In [28]:
closestGene1[closestGene1[3].isin(['EH38E2836060','EH38E2836065','EH38E1382733','EH38E2836036','EH38E2836028','EH38E2835994'])]

,0,1,2,3,11,12,13,14,15
226118,chr1,147539962,147540170,EH38E2835994,chr1,147541501,147626216,ENSG00000116128,1332
226119,chr1,147539962,147540170,EH38E2835994,chr1,147541501,147626216,ENSG00000116128,1332
226120,chr1,147539962,147540170,EH38E2835994,chr1,147541501,147626216,ENSG00000116128,1332
226121,chr1,147539962,147540170,EH38E2835994,chr1,147541501,147626216,ENSG00000116128,1332
226178,chr1,147599385,147599535,EH38E2836028,chr1,147608331,147610187,ENSG00000289419,8797
226187,chr1,147611213,147611484,EH38E2836036,chr1,147611590,147611796,ENSG00000288626,107
226225,chr1,147627788,147627955,EH38E1382733,chr1,147541501,147626216,ENSG00000116128,1573
226226,chr1,147627788,147627955,EH38E1382733,chr1,147541501,147626216,ENSG00000116128,1573
226227,chr1,147627788,147627955,EH38E1382733,chr1,147541501,147626216,ENSG00000116128,1573
226228,chr1,147627788,147627955,EH38E1382733,chr1,147541501,147626216,ENSG00000116128,1573


In [29]:
closestGene1.tail()

,0,1,2,3,11,12,13,14,15
4481558,chrY,26409363,26409710,EH38E3951258,.,-1,-1,.,-1
4481559,chrY,26563051,26563261,EH38E3951263,.,-1,-1,.,-1
4481560,chrY,26661142,26661310,EH38E3951264,.,-1,-1,.,-1
4481561,chrY,26670948,26671287,EH38E3951269,.,-1,-1,.,-1
4481562,chrY,26671292,26671478,EH38E3951270,.,-1,-1,.,-1


In [30]:
closestGene1[closestGene1[3].isin(['EH38E1310568','EH38E1310568','EH38E1310568','EH38E1310568','EH38E2777292'])]

,0,1,2,3,11,12,13,14,15
1794,chr1,1168609,1168805,EH38E1310568,chr1,1169005,1169087,ENSG00000198976,201
2555,chr1,1291182,1291527,EH38E2777292,chr1,1292390,1309609,ENSG00000131584,864
2556,chr1,1291182,1291527,EH38E2777292,chr1,1292390,1309609,ENSG00000131584,864
2557,chr1,1291182,1291527,EH38E2777292,chr1,1292390,1309609,ENSG00000131584,864
2558,chr1,1291182,1291527,EH38E2777292,chr1,1292390,1309609,ENSG00000131584,864
2559,chr1,1291182,1291527,EH38E2777292,chr1,1292390,1309609,ENSG00000131584,864
2560,chr1,1291182,1291527,EH38E2777292,chr1,1292390,1309609,ENSG00000131584,864
2561,chr1,1291182,1291527,EH38E2777292,chr1,1292390,1309609,ENSG00000131584,864
2562,chr1,1291182,1291527,EH38E2777292,chr1,1292390,1309609,ENSG00000131584,864
2563,chr1,1291182,1291527,EH38E2777292,chr1,1292390,1309609,ENSG00000131584,864


In [51]:
closestGene1[closestGene1[3] =='EH38E2789351']

,0,1,2,3,11,12,13,14,15
10580,chr1,16181671,16181834,EH38E2789351,chr1,16197854,16198357,ENSG00000234166,16021
10581,chr1,16181671,16181834,EH38E2789351,chr1,16197854,16212652,ENSG00000142632,16021


In [32]:
closestGene1 = closestGene1[closestGene1[14].str.contains('ENSG')]

In [33]:
closestGene1 = closestGene1[closestGene1[3].str.contains('EH')]

In [34]:
closestGene1 = closestGene1.drop_duplicates().reset_index(drop = True)

In [35]:
closestGene1.to_csv('tempFinaldistance.bed',sep='\t',header=None,index=False)

In [36]:
len(closestGene1.nunique())

9

In [37]:
df22 = pd.read_csv('../../../storage/data/dna/human/hg38/hg38.agp.gz',sep="\t",header=None)

In [38]:
df22.head(10)

,0,1,2,3,4,5,6,7,8
0,chr1,1,10000,1,N,10000,telomere,no,na
1,chr1,10001,10615,2,F,AP006221.1,36117,36731,-
2,chr1,10616,177417,3,F,AL627309.15,103,166904,+
3,chr1,177418,207666,4,F,FO538757.3,2001,32249,+
4,chr1,207667,257666,5,N,50000,contig,no,na
5,chr1,257667,297968,6,F,AP006222.1,1,40302,+
6,chr1,297969,347968,7,N,50000,contig,no,na
7,chr1,347969,501617,8,F,AL732372.15,1,153649,-
8,chr1,501618,535988,9,P,FO681485.2,1,34371,-
9,chr1,535989,585988,10,N,50000,contig,no,na


In [39]:
df22[(df22[5] == 'ENSG00000162836')]

,0,1,2,3,4,5,6,7,8


In [40]:
df32 = pd.read_csv('mart_export.txt.gz',sep="\t")

In [41]:
df32.head()

,Gene stable ID,Transcript stable ID,HGNC ID,HGNC symbol,Gene start (bp),Gene end (bp),Chromosome/scaffold name
0,ENSG00000227568,ENST00000458433,HGNC:39634,SNX18P26,18807279,18808018,13
1,ENSG00000230931,ENST00000446657,HGNC:39676,GXYLT1P1,18724415,18724904,13
2,ENSG00000204718,ENST00000330492,HGNC:39834,CNN2P12,18726706,18727414,13
3,ENSG00000231238,ENST00000652806,HGNC:42667,LINC00349,18538880,18551214,13
4,ENSG00000231238,ENST00000448748,HGNC:42667,LINC00349,18538880,18551214,13


In [42]:
df32[(df32['Gene stable ID']== 'ENSG00000162836') ]

,Gene stable ID,Transcript stable ID,HGNC ID,HGNC symbol,Gene start (bp),Gene end (bp),Chromosome/scaffold name
163310,ENSG00000162836,ENST00000609196,HGNC:29609,ACP6,147629652,147670524,1
163311,ENSG00000162836,ENST00000583509,HGNC:29609,ACP6,147629652,147670524,1
163312,ENSG00000162836,ENST00000460583,HGNC:29609,ACP6,147629652,147670524,1
163313,ENSG00000162836,ENST00000613673,HGNC:29609,ACP6,147629652,147670524,1
163314,ENSG00000162836,ENST00000487562,HGNC:29609,ACP6,147629652,147670524,1
163315,ENSG00000162836,ENST00000392988,HGNC:29609,ACP6,147629652,147670524,1
163316,ENSG00000162836,ENST00000620634,HGNC:29609,ACP6,147629652,147670524,1
163317,ENSG00000162836,ENST00000611629,HGNC:29609,ACP6,147629652,147670524,1
163318,ENSG00000162836,ENST00000614551,HGNC:29609,ACP6,147629652,147670524,1
163319,ENSG00000162836,ENST00000493129,HGNC:29609,ACP6,147629652,147670524,1


In [43]:
df32[(df32['Gene start (bp)']== 147629652) ]

,Gene stable ID,Transcript stable ID,HGNC ID,HGNC symbol,Gene start (bp),Gene end (bp),Chromosome/scaffold name
163310,ENSG00000162836,ENST00000609196,HGNC:29609,ACP6,147629652,147670524,1
163311,ENSG00000162836,ENST00000583509,HGNC:29609,ACP6,147629652,147670524,1
163312,ENSG00000162836,ENST00000460583,HGNC:29609,ACP6,147629652,147670524,1
163313,ENSG00000162836,ENST00000613673,HGNC:29609,ACP6,147629652,147670524,1
163314,ENSG00000162836,ENST00000487562,HGNC:29609,ACP6,147629652,147670524,1
163315,ENSG00000162836,ENST00000392988,HGNC:29609,ACP6,147629652,147670524,1
163316,ENSG00000162836,ENST00000620634,HGNC:29609,ACP6,147629652,147670524,1
163317,ENSG00000162836,ENST00000611629,HGNC:29609,ACP6,147629652,147670524,1
163318,ENSG00000162836,ENST00000614551,HGNC:29609,ACP6,147629652,147670524,1
163319,ENSG00000162836,ENST00000493129,HGNC:29609,ACP6,147629652,147670524,1
